In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skewnorm, t
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Function to calculate parameters for skew normal from moments
def moments_to_skew_normal(mean, variance, skewness):
    scale = np.sqrt(variance / (1 - (2 * (skewness / np.sqrt(1 + skewness**2))**2 / np.pi)))
    alpha = skewness / np.sqrt(1 - skewness**2)
    location = mean - scale * (np.sqrt(2 / np.pi) * alpha / np.sqrt(1 + alpha**2))
    return location, scale, alpha


# Function to generate skew normal data
def generate_skew_normal(size, mean, variance, skewness):
    location, scale, alpha = moments_to_skew_normal(mean, variance, skewness)
    data = skewnorm.rvs(alpha, loc=location, scale=scale, size=size)
    return data

# Simulation parameters
n_samples = 252  # Number of log returns per dataset
initial_price = 100  # Initial price for converting log returns to prices
nu = 2.5  # Degrees of freedom for Student-t

mean_return_gauss = 0.1
mean_return = 0
variance = 0.25**2

# Moments for each dataset
moments = {
    "gaussian": (mean_return_gauss/n_samples, variance/n_samples, 0.0),
    "skew_normal_neg": (mean_return/n_samples, variance/n_samples, -0.9),
    "t_student_neut": (mean_return/n_samples, variance/n_samples, 0.0),
}

# Generate datasets

gaussian_data = np.random.normal(loc=moments["gaussian"][0], scale=np.sqrt(moments["gaussian"][1]), size=n_samples)
skew_normal_pos = generate_skew_normal(n_samples, *moments["skew_normal_pos"])
skew_normal_neg = generate_skew_normal(n_samples, *moments["skew_normal_neg"])
skew_student_pos = generate_skew_student_t(n_samples, *moments["skew_student_pos"], nu)
skew_student_neg = generate_skew_student_t(n_samples, *moments["skew_student_neg"], nu)

# Convert log returns to prices
def log_returns_to_prices(log_returns, initial_price):
    return initial_price * np.exp(np.cumsum(log_returns))

prices_gaussian = log_returns_to_prices(gaussian_data, initial_price)
prices_skew_normal_pos = log_returns_to_prices(skew_normal_pos, initial_price)
prices_skew_normal_neg = log_returns_to_prices(skew_normal_neg, initial_price)
prices_skew_student_pos = log_returns_to_prices(skew_student_pos, initial_price)
prices_skew_student_neg = log_returns_to_prices(skew_student_neg, initial_price)

# Save datasets to CSV files
def save_to_csv(data, filename):
    df = pd.DataFrame(data, columns=["Log_Return"])
    df.to_csv(filename, index=False)

# File names
save_to_csv(gaussian_data, "gaussian_data.csv")
save_to_csv(skew_normal_pos, "skew_normal_positive.csv")
save_to_csv(skew_normal_neg, "skew_normal_negative.csv")
save_to_csv(skew_student_pos, "skew_student_positive.csv")
save_to_csv(skew_student_neg, "skew_student_negative.csv")

# Plot histograms for comparison
plt.figure(figsize=(12, 8))

plt.hist(gaussian_data, bins=30, alpha=0.5, label="Gaussian", density=True)
plt.hist(skew_normal_pos, bins=30, alpha=0.5, label="Skew Normal Positive", density=True)
plt.hist(skew_normal_neg, bins=30, alpha=0.5, label="Skew Normal Negative", density=True)
plt.hist(skew_student_pos, bins=30, alpha=0.5, label="Skew Student Positive", density=True)
plt.hist(skew_student_neg, bins=30, alpha=0.5, label="Skew Student Negative", density=True)

plt.title("Histogram Comparison of Simulated Datasets")
plt.xlabel("Log Return")
plt.ylabel("Density")
plt.legend()
plt.grid()
plt.show()

# Plot price time series
plt.figure(figsize=(12, 8))

plt.plot(prices_gaussian, label="Gaussian")
plt.plot(prices_skew_normal_pos, label="Skew Normal Positive")
plt.plot(prices_skew_normal_neg, label="Skew Normal Negative")
plt.plot(prices_skew_student_pos, label="Skew Student Positive")
plt.plot(prices_skew_student_neg, label="Skew Student Negative")

plt.title("Price Time Series Comparison")
plt.xlabel("Time Step")
plt.ylabel("Price")
plt.legend()
plt.grid()
plt.show()

print("Datasets generated, saved to CSV files, histograms plotted, and price time series plotted.")
